The current setup allows to execute PySpark operations on CERN Hadoop and Spark clusters. This notebook illustrates the use of Spark in SWAN to analyze the monitoring data available on HDFS and plots a heatmap of loadAvg across machines in a particular service.

### Connect to the cluster

To connect to a cluster, click on the star button on the top and follow the instructions
* The star button only appears if you have selected a SPARK cluster in the configuration
* The star button is active after the notebook kernel is ready

### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [3]:
df = spark.read.json("/user/rucio01/traces/traces.2018*")

### check the structure

In [4]:
#df.printSchema()

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [5]:
df.head()

Row(_corrupt_record=None, account=None, appid=u'3772225596', catStart=1514761197.84869, clientState=u'DONE', dataset=u'panda.um.data17_13TeV.00340030.physics_Main.deriv.DAOD_BPHY6.f889_m1902_p3267_tid12608781_00.169436420', datasetName=None, datasetScope=None, dataset_scope=None, duid=None, eventType=u'get_sm', eventVersion=u'pilot3', filename=u'DAOD_BPHY6.12608781._000245.pool.root.1.panda.um_002', filesize=u'57392255', guid=u'4E82F48B91AAC946B8E897E3CCFE62C7', hostname=u'c-8-23.aglt2.org', ip=u'192.41.230.149', localSite=u'AGLT2_DATADISK', name=None, pq=u'AGLT2_SL6', protocol=u'lsm', relativeStart=1514761197.850914, remoteSite=u'AGLT2_DATADISK', scope=u'panda', stateReason=u'OK', suspicious=u'0', taskid=u'12608781', timeEnd=1514761199.339337, timeStart=1514761014.277093, traceId=u'7d679dc663e44880a812b0d709e9b7ee', traceIp=u'::ffff:192.41.230.149', traceTimeentry=u'2017-12-31T23:00:00.007341', traceTimeentryUnix=1514761200.007341, transferEnd=None, transferStart=1514761197.850914, ur

### Create temporary table view

In [6]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Use PySpark to query download events and split dataset DID into new features

In [7]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [8]:
df_ps = spark.sql("SELECT dataset, from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') AS dt \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND (dataset LIKE 'data%' OR dataset LIKE 'mc%') AND from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') >= date_add(current_date(), -30)\
                               ORDER BY 1")\

In [9]:
df_ps.show(n=5)

+--------------------+----------+
|             dataset|        dt|
+--------------------+----------+
|data10_hi.0016992...|2018-04-27|
|data10_hi:data10_...|2018-04-28|
|data10_hi:data10_...|2018-04-28|
|data10_hi:data10_...|2018-04-28|
|data10_hi:data10_...|2018-04-28|
+--------------------+----------+
only showing top 5 rows



In [10]:
split_col = split(df_ps['dataset'], "\.")
df_ps = df_ps.withColumn('project', split_col.getItem(0))
df_ps = df_ps.withColumn('run_number', split_col.getItem(1))
df_ps = df_ps.withColumn('stream_name', split_col.getItem(2))
df_ps = df_ps.withColumn('prod_step', split_col.getItem(3))
df_ps = df_ps.withColumn('datatype', split_col.getItem(4))
df_ps = df_ps.withColumn('dataset_version', split_col.getItem(5))
df_ps.show(n=5)


+--------------------+----------+-------------------+----------+------------+---------+--------+--------------------+
|             dataset|        dt|            project|run_number| stream_name|prod_step|datatype|     dataset_version|
+--------------------+----------+-------------------+----------+------------+---------+--------+--------------------+
|data10_hi.0016992...|2018-04-27|          data10_hi|  00169927|physics_bulk|     ROOT| NTUP_HI|          r2111_p456|
|data10_hi:data10_...|2018-04-28|data10_hi:data10_hi|  00169927|physics_bulk|     ROOT| NTUP_HI|r2111_p456_tid289...|
|data10_hi:data10_...|2018-04-28|data10_hi:data10_hi|  00169927|physics_bulk|     ROOT| NTUP_HI|r2111_p456_tid289...|
|data10_hi:data10_...|2018-04-28|data10_hi:data10_hi|  00169927|physics_bulk|     ROOT| NTUP_HI|r2111_p456_tid289...|
|data10_hi:data10_...|2018-04-28|data10_hi:data10_hi|  00169927|physics_bulk|     ROOT| NTUP_HI|r2111_p456_tid289...|
+--------------------+----------+-------------------+---